# TODO

- vector_store.ipynb
    - data/소설들.pdf 들 Vector store에 저장.
        - chroma, pinecone (알아서)
        - metadata 
            - title: 소설제목
            - author: 저자
            - full_text: 소설 전체 내용
        - Document(page_content="split된 내용", metadata={})
        - page_content는 조회할 때 사용할 embedding vector를 생성할 때 사용될 내용.
        - llm 모델에 전송할 context는 metadata의 full_text

- app.py
    - streamlit을 이용해서 서비스하는 application
    - vector_store.ipynb에서 구축한 DB를 이용해 질문과 관련된 내용들을 
      조회해서 llm에 전송하고 그 결과를 채팅창에 출력.
    - memory 기능은 사용하지 않는다.

# 소설들을 Vector Store에 저장

## Load
- `data/*.pdf` 파일들을 모두 load

- 각 소설들을 split 한 뒤 vector store에 저장한다.   
- metadata:
    - title: 소설제목
    - author: 저자
    - full_text: 소설 전체 내용

In [8]:
glob("data/*.pdf")
"----".join(["a", "b","안녕"])

'a----b----안녕'

In [9]:
import re
import os
import config
from glob import glob

from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document

from dotenv import load_dotenv

load_dotenv()

# 설정 정보 읽기
CHUNK_SIZE = config.chunk_size
CHUNK_OVERLAP = config.chunk_overlap

MODEL_NAME  = config.model_name
EMBEDDING_NAME = config.embedding_name

COLLECTION_NAME = config.collection_name
PERSIST_DIRECTORY = config.persist_directory

# 소설 파일들 경로 조회
path_list = glob("data/*.pdf")

document_list = []

# load -> 전처리 -> split -> metadata 추가 
for path in path_list:
    # Document Load
    loader = PyMuPDFLoader(path)
    load_docs = loader.load() #list[Document]

    # 전처리 - 페이지 번호 제거, 🙝🙟 를 \n\n 으로 변경
    full_text = [doc.page_content for doc in load_docs] 
    full_text = ''.join(full_text) # page content들을 문자열로 합치기.
    full_text = full_text.replace("🙝🙟", "\n\n")
    full_text = re.sub(r"\d+\n", " ", full_text)   # 페이지 번호 제거
    

    # Split
    splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        model_name=MODEL_NAME,
        chunk_size=CHUNK_SIZE,
        chunk_overlap=CHUNK_OVERLAP,
    )
    
    # splitter.split_documents([Document, Document,. ....]) -> list[Document]
    docs = splitter.split_text(full_text) # 문자열 split -> list[str]

    # Metadata 생성
    author = os.path.splitext(os.path.basename(path))[0].split('_')[-1]
    metadata = {
        "title":load_docs[0].metadata["title"],
        "author": author,
        "full_text":full_text
    }

    # Document 생성
    for doc in docs: # docs: split된 소설 내용들. list[str]
        _doc = Document(metadata=metadata, page_content=doc)
        document_list.append(_doc)

print(len(document_list))

107


In [11]:
print(document_list[2].metadata)

{'title': '금 따는 콩밭', 'author': '김유정', 'full_text': " 금 따는 콩밭\nExported from Wikisource on 2024년 11월 24일\n 위키백과\n위키백과에 이 글\n과 관련된\n자료가 있습니다.\n금 따는 콩밭\n\n\n\n땅속 저 밑은 늘 음침하\n다.\n고달픈 간드렛불, 맥없이\n푸르끼하다.\n밤과 달라서 낮엔 되우 흐릿하였다.\n겉으로 황토 장벽으로 앞뒤좌우가 콕 막힌 좁직한 구뎅이.\n흡사히 무덤 속같이 귀중중하다. 싸늘한 침묵, 쿠더브레한\n흙내와 징그러운 냉기만이 그 속에 자욱하다.\n곡괭이는 뻔질 흙을 이르집는다. 암팡스러이 내려쪼며,\n퍽 퍽 퍼억.\n이렇게 메떨어진 소리뿐. 그러나 간간 우수수 하고 벽이 헐\n린다.\n영식이는 일손을 놓고 소맷자락을 끌어당기어 얼굴의 땀을\n훑는다. 이놈의 줄이 언제나 잡힐는지 기가 찼다. 흙 한줌을\n집어 코밑에 바짝 들여대고 손가락으로 샅샅이 뒤져본다. 완\n연히 버력은 좀 변한 듯싶다. 그러나 불통버력이 아주 다 풀\n린 것도 아니었다. 밀똥버력이라야 금이 온다는데 왜 이리\n안 나오는지.\n곡괭이를 다시 집어든다. 땅에 무릎을 꿇고 궁뎅이를 번쩍\n든 채 식식거린다. 곡괭이는 무작정 내려찍는다. 바닥에서\n 물이 스미어 무르팍이 흔건히 젖었다. 굿엎은 천판에서 흙방\n울은 내리며 목덜미로 굴러든다. 어떤 때에는 웃벽의 한쪽이\n떨어지며 등을 탕 때리고 부서진다.\n그러나 그는 눈도 하나 깜짝하지 않는다. 금을 캔다고 콩밭\n하나를 다 잡쳤다. 약이 올라서 죽을둥 살둥 눈이 뒤집힌 이\n판이다. 손바닥에 침을 탁 뱉고 곡괭이 자루를 한번 꼰아잡\n더니 쉴 줄 모른다.\n등뒤에서는 흙 긁는 소리가 드윽드윽 난다. 아직도 버력을\n다 못 친 모양. 이 자식이 일을 하나 시졸 하나. 남은 속이 바\n직바직 타는데 웬 뱃심이 이리도 좋아.\n영식이는 살기 띤 시선으로 고개를 돌렸다. 암 말 없이 수재\n를 노려본다. 그제야 꾸물꾸물 바지게에 흙을 담고 등에 

## Vector Store 저장
- Chroma Vector Store에 저장

In [12]:
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma


embedding_model = OpenAIEmbeddings(
    model=EMBEDDING_NAME
)

# [생성]/연결 + document들 추가가
vector_store = Chroma.from_documents(
    documents=document_list,
    embedding=embedding_model,
    collection_name=COLLECTION_NAME,
    persist_directory=PERSIST_DIRECTORY
)

# 확인

In [13]:
# 자정 개수 확인
vector_store._collection.count()

107

In [14]:
###########################
# 검색

query = "현진건의 소설들의 특징은 어떤 것이 있을까?" 
query = "운수좋은 날에서 주인공은 왜 괴상하게도 운수가 좋다고 느꼈을까?"
query = "허 생원은 어느 장으로 가서 장사를 할 예정인가?"
query = "이효석의 메밀꽃 필 무렵의 주인공은 누구인가?"
query = "벙어리 삼룡이는 화자가 몇살 때 일인가?"
query = "삼룡이의 주인 아들의 색시는 몇살인가?"

result = vector_store.similarity_search(
    query,
    k=5,
    # filter={"author": "현진건"}
)

In [15]:
for res in result:
    print(res.metadata['title'], res.metadata['author'], sep=" | ")
    print(res.page_content)
    print("=====================================")

벙어리 삼룡이 | 나도향
이리하여 색시는 시집오던 날부터 팔자 한탄을 하고서 날마
다 밤마다 우는 사람이 되었다.
울며는 요사스럽다고 때린다. 또 말이 없으면, 빙충맞다고
친다. 이리하여 그 집에는 평화스러운 날이 하루도 없었다.
이것을 날마다 보는 사람 가운데 알 수 없는 의혹을 품게 된
사람이 하나 있으니 그는 곧 벙어리 삼룡이었다.
그렇게 예쁘고 유순하고 그렇게 얌전한 벙어리의 눈으로 보
아서는 감히 손도 대지 못할 만치 선녀 같은 색시를 때리는
 것은 자기의 생각으로는 도저히 풀 수 없는 의심이다.
보기에는 황홀하고 건드리기도 황홀할 만치 숭고한 여자를
그렇게 하대한다는 것은 너무나 세상에 있지 못할 일이다.
자기는 주인 새서방에게 개나 돼지같이 얻어맞는 것이 마땅
한 이상으로 마땅하지마는 선녀와 짐승의 차가 있는 색시와
자기가 똑같이 얻어맞는 것은 너무 무서운 일이다. 어린 주
인이 천벌이나 받지 않을까 두렵기까지 하였다.
어떠한 달밤, 사면은 고요적막하고 별들은 드문드문 눈들만
깜박이며 반달이 공중에 뚜렷이 달려 있어 수은으로 세상을
깨끗하게 닦아낸 듯이 청명한데 삼룡이는 검둥개 등을 쓰다
듬으며 밖 마당 멍석 위에 비슷이 드러누워 하늘을 쳐다보며
생각하여 보았다.
주인 색시를 생각하면 공중에 있는 달보다도 더 곱고 별들보
다도 더 깨끗하였다. 주인 색시를 생각하면 달이 보이고 별
이 보이었다. 삼라만상을 씻어내는 은빛보다도 더 흰 달이나
별의 광채보다도 그의 마음이 아름답고 부드러운 듯하였다.
마치 달이나 별이 땅에 떨어져 주인 새아씨가 된 것도 같고
주인 새아씨가 하늘에 올라가면 달이 되고 별이 될 것 같았
다.
 | 황순원
“그럼, 참외 맛도 좋지만 수박 맛은 더 좋다.”
“하나 먹어 봤으면.”
소년이 참외 그루에 심은 무우밭으로 들어가, 무우 두 밑을 뽑아 왔다. 아직 밑이 덜 들어 
있었다. 잎을 비틀어 팽개친 후, 소녀에게 한개 건넨다. 그리고는 이렇게 먹어야 한다는 듯
이, 먼저 대강이를 한 입 베물어 낸 다음, 손톱으로 한 돌이 껍질을

In [19]:
from textwrap import dedent
s = """
        a
        g
        c
"""
print(s)
print(dedent(s))


        a
        g
       c


 a
 g
c

